In [ ]:
import pandas as pd
import numpy as np
import joblib
import re
import string
#train test split(the data is splitted into training and testing data)
from sklearn.model_selection import train_test_split
#it is used to extract features from text data(convert text into numbers)
from sklearn.feature_extraction.text import TfidfVectorizer
#the model we will be using in this mp
from sklearn.linear_model import LogisticRegression
#this is used to measure accuracy(f1 score, precision, recall)of the model
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report   
#this helps in removing stop words(words which are not useful-commonly used words)
from nltk.corpus import stopwords
#this helps in stemming(removing suffixes and prefixes)
from nltk.stem.porter import PorterStemmer



In [18]:


stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

merged_data['text'] = merged_data['text'].apply(remove_stopwords)

NameError: name 'merged_data' is not defined

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\muaaz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
true=pd.read_csv('True.csv')
fake=pd.read_csv('Fake.csv')

In [ ]:
true['label']=1
fake['label']=0

In [ ]:
true.loc[2,'title']

"Senior U.S. Republican senator: 'Let Mr. Mueller do his job'"

In [20]:
merged_data=pd.concat([true,fake],axis=0)

In [21]:
merged_data.sample(20)

,title,text,subject,date
20264,Trump to call Mexico's Pena Nieto in earthquak...,WASHINGTON (Reuters) - U.S. President Donald T...,worldnews,"September 12, 2017"
8653,Former Ku Klux Klan leader Duke runs for U.S. ...,"BATON ROUGE, La. (Reuters) - David Duke, a for...",politicsNews,"July 22, 2016"
12253,BREAKING: CONDI RICE Meets With Mike Pence In ...,How many outspoken never-Trumper s is Donald...,politics,"Dec 1, 2016"
16894,Germany will await prosecution 'ok' before del...,BERLIN (Reuters) - Germany will wait for Israe...,worldnews,"October 20, 2017"
18723,Putin complains Russian media abroad face unac...,MOSCOW (Reuters) - President Vladimir Putin to...,worldnews,"September 29, 2017"
17848,German police arrest man after posting photo o...,FRANKFURT (Reuters) - German authorities arres...,worldnews,"October 10, 2017"
4017,WATCH: CNN Host Calls Out Trump Supporter For...,This election is already the nastiest one on r...,News,"October 28, 2016"
15525,BREAKING: OBAMA’S RACE WAR PART II…BRAWL BREAK...,A defender of the Confederate flag who says th...,politics,"Jun 30, 2015"
7699,Highly Ranked Draft Prospect Asked By NFL Coa...,It what can only be seen as completely unaccep...,News,"March 4, 2016"
14881,Tokyo governor quits as head of conservative o...,TOKYO (Reuters) - The governor of the Japanese...,worldnews,"November 14, 2017"


In [22]:
merged_data = merged_data.drop(['subject', 'date'], axis=1,)

In [23]:
merged_data = merged_data.sample(frac=1).reset_index(drop=True)

In [24]:
merged_data.head(12)

,title,text
0,Coke Just Stuck It To Trump’s Bigotry With Th...,The whole nation is ashamed of the open and ov...
1,Trump tweets he raised $13 million in 24 hours...,WASHINGTON (Reuters) - Republican Party presid...
2,California senators seek methane leak task for...,WASHINGTON (Reuters) - The two U.S. Senators r...
3,Trump’s Response To Special Counsel Shows Jus...,The Department of Justice announced on Wednesd...
4,Trump picks 'Death by China' author for trade ...,(Reuters) - U.S. President-elect Donald Trump ...
5,Syria toxic gas inquiry to end after Russia ag...,UNITED NATIONS (Reuters) - An international in...
6,"Spanish PM, in Catalonia, calls for big turnou...",BARCELONA - Prime Minister Mariano Rajoy urged...
7,"Desperate For Guests, Fox News’ O’Reilly Repl...",Until today s episode of Tucker Carlson Tonigh...
8,At least eight dead amid Cameroon Anglophone p...,"BUEA/BAMENDA, Cameroon (Reuters) - Soldiers sh..."
9,Xi says China has prevented Taiwan independenc...,BEIJING (Reuters) - Chinese President Xi Jinpi...


In [ ]:

ps = PorterStemmer()

def stem_text(text):
    words = text.split()
    stemmed_words = [ps.stem(word) for word in words]
    return ' '.join(stemmed_words)

merged_data['text'] = merged_data['text'].apply(stem_text)

In [ ]:
from tqdm import tqdm

tqdm.pandas()
merged_data['text'] = merged_data['text'].progress_apply(stem_text)

100%|██████████| 44898/44898 [05:12<00:00, 143.84it/s]


In [ ]:
merged_data

NameError: name 'merged_data' is not defined

In [ ]:

merged_data['text'] = merged_data['text'].str.lower()


NameError: name 'merged_data' is not defined

In [ ]:
merged_data

,title,text,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,1
...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,0


In [ ]:
#removes brackets

merged_data['text'] = merged_data['text'].str.replace(r"[\(\)\[\]\{\}]", "", regex=True)

In [ ]:
#remove any URL
merged_data['text']=merged_data['text'].str.replace(r"http\S+|www\S+", "", regex=True)



In [ ]:
# removes everything except alphabets
merged_data['text'] = merged_data['text'].str.replace(r"[^a-zA-Z\s]", "", regex=True)

In [ ]:
merged_data

NameError: name 'merged_data' is not defined

In [ ]:
x=merged_data['text']
y=merged_data['label']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=40)

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
xv_train = vectorizer.fit_transform(x_train)
xv_test = vectorizer.transform(x_test)




In [ ]:
lreg=LogisticRegression()
lreg.fit(xv_train,y_train)

LogisticRegression()

In [ ]:
pred=lreg.predict(xv_test)
lreg.score(xv_test,y_test)

0.9900890868596882

In [ ]:
print(accuracy_score(y_test,pred))

0.9900890868596882


In [ ]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4684
           1       0.99      0.99      0.99      4296

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [ ]:
joblib.dump(vectorizer,'vectorizer.jb')
joblib.dump(lreg,'model.jb')


['model.jb']